# AI Red Teaming Agent for Copilot Studio Agents

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of **Copilot Studio-created agents* against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
First, if you have an Azure subscription, create an [Azure AI hub](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources) then [create an Azure AI project](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources). AI projects and Hubs can be served within a private network and are compatible with private endpoints. You **do not** need to provide your own LLM deployment as the AI Red Teaming Agent hosts adversarial models for both simulation and evaluation of harmful content and connects to it via your Azure AI project.

In order to upload your results to Azure AI Foundry:
- Your AI Foundry project must have a connection (*Connected Resources*) to a storage account with `Microsoft Entra ID` authentication enabled.
- Your AI Foundry project must have the `Storage Blob Data Contributor` role in the storage account.
- You must have the `Storage Blob Data Contributor` role in the storage account.
- You must have network access to the storage account.

For more information see: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent

**Recommendation**: This notebook shares most of the prerequisites with [AI_RedTeaming](./AI_RedTeaming.ipynb), so we recommend reviewing this one first.


### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity openai azure-ai-projects
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

## Setting up Copilot Studio Client Library (Preview)

Copilot Studio Client requires some libraries currently **in preview**, that require installing from a test repository. 

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install msal-extensions

pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-core
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-authorization
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-connector
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-client
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-builder
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-authentication-msal
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-copilotstudio-client
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-hosting-aiohttp
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ microsoft-agents-storage
```

### Imports

In [ ]:
from typing import Optional, Dict, Any
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# Copilot Studio dependencies (Preview)
# Requires Activity class at version >= https://github.com/microsoft/Agents-for-python/pull/52
from microsoft.agents.core.models import ActivityTypes

# This class is a wrapper for the Copilot Studio client class currently in Preview.
from src.CopilotStudioClient import McsCopilotClient, McsConnectionSettings

### Login with your required Azure Credentials

Ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

Configure the `credential` object with a different AzureCredential type if this is a requirement for your environment.

In [ ]:
# Azure Credential imports
from azure.identity import AzureCliCredential

!az login

# Initialize Azure credentials
credential = AzureCliCredential()

### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

Set these variables by creating an `.env` file in your project's root folder.

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

For reference, here's an example of what your populated environment variables should look like:

```
# Azure OpenAI
AZURE_OPENAI_API_KEY="your-api-key-here"
AZURE_OPENAI_ENDPOINT="https://endpoint-name.cognitiveservices.azure.com/"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4"
AZURE_OPENAI_API_VERSION="2024-12-01-preview"

# Azure AI Project
AZURE_PROJECT_ENDPOINT="https://your-aifoundry-endpoint-name.services.ai.azure.com/api/projects/yourproject-name"
```

In [ ]:
# Azure AI Project information
azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")

# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")  # e.g., "gpt-4"
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  # e.g., "your-api-key"
azure_openai_api_version = os.environ.get("OPENAI_API_VERSION")  # Use the latest API version

These variables are required to connect to your Copilot Studio agent endpoint

**From Entra ID** ([Configure user authentication with Microsoft Entra ID](https://learn.microsoft.com/en-us/microsoft-copilot-studio/configuration-authentication-azure-ad)):

```
# App registration
TENANT_ID="your-tenant-guid"
APP_CLIENT_ID="your-app-registration-client-guid" 
```

**From Copilot Studio > Settings > Advanced > Metadata** ([Integrate with web or native apps using Microsoft 365 Agents SDK](https://learn.microsoft.com/en-us/microsoft-copilot-studio/publication-integrate-web-or-native-app-m365-agents-sdk)):

```
# Environment ID
ENVIRONMENT_ID="your-environment-guid"
# Schema Name
AGENT_IDENTIFIER="xy0z_YourAgentName"
```

In [ ]:
tenant_id = os.environ.get("TENANT_ID")
app_client_id = os.environ.get("APP_CLIENT_ID")

environment_id = os.environ.get("ENVIRONMENT_ID")
agent_identifier = os.environ.get("AGENT_IDENTIFIER")

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content.
You can also use `AttackStrategy.Compose()` to layer two strategies in one attack
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: Different difficultly levels of attacks
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

## Test connectivity to your Copilot Studio client

In this basic example a Copilot Client is created and a simple prompt is used for testing connectivity with the agent endpoint.

In [ ]:
# Initialize the Copilot Studio Client helper
client = McsCopilotClient()

# Start a conversation with the Copilot Studio agent
await client.start_conversation_async()

# Ask a question to the Copilot Studio agent
activities = await client.ask_question_async("Tell me a joke about a Copilot Studio low-code developer.")

# Print the activities returned by the Copilot Studio agent
message = "".join(activity.text for activity in activities if activity.type == ActivityTypes.message)
print(message)

## Advanced Example: Using a Copilot Studio Client in a Callback Function

Connectivity to Copilot Studio endpoint is wrapped into a callback function that handles input and output manually for evaluation. For more examples and detailed explanation, please see [AI_RedTeaming](./AI_RedTeaming.ipynb) notebook.

In [ ]:
# Define a callback that uses a Copilot Studio Agent to generate responses
async def copilot_studio_agent_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Create a connection settings object for the Copilot Studio client
        connection_settings = McsConnectionSettings(
            tenant_id=tenant_id,
            app_client_id=app_client_id,
            environment_id=environment_id,
            agent_identifier=agent_identifier,
        )

        # Initialize the Copilot Studio Client helper
        client = McsCopilotClient(connection_settings=connection_settings)

        # Start a conversation with the Copilot Studio agent (Copilot's welcome message)
        activities = await client.start_conversation_async()

        # Ask a question to the Copilot Studio agent
        activities = await client.ask_question_async(latest_message)
        messages = "".join(activity.text for activity in activities if activity.type == ActivityTypes.message)

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": messages, "role": "assistant"}

    except Exception as e:
        print(f"Error calling Copilot Studio Agent: {e!s}")
        formatted_response = f"I encountered an error and couldn't process your request: {e!s}"

    return {"messages": [formatted_response]}

Due to the **content management** and **threat detection policies** in Copilot Studio, some of the probing prompts used by PyRIT will trigger errors as Copilot Studio will refuse these prompts. Some of these errors will show as exceptions in this notebook's outcome.

This behavior is expected.

In [ ]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for one risk category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=copilot_studio_agent_callback,
    scan_name="Advanced-Callback-Scan",
    application_scenario="Test Evaluation for Copilot Studio",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

## Bring your own objectives: Using your own prompts as objectives for RedTeam

Below we demonstrate how to use your own prompts as objectives for a `RedTeam` scan. You can see the required format for prompts under `.\data\prompts.json`. Note that when bringing your own prompts, the supported `risk-type`s are `violence`, `sexual`, `hate_unfairness`, and `self_harm`. The number of prompts you specify will be the `num_objectives` used in the scan. 

In [ ]:
path_to_prompts = ".\data\prompts.json"

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
)

In [ ]:
custom_red_team_result = await custom_red_team.scan(
    target=copilot_studio_agent_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
    output_path="Custom-Prompt-Scan.json",
)

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of an **agent created in Copilot Studio**, including every customization and external data retrieval and actions included in the final result, testing the product the users will finally consume.

We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 